In [3]:
import numpy as np
import pandas as pd 
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from category_encoders.count import CountEncoder


NUM_FOLDS=5
RANDOM_STATE = 800


#     ░░░░░░░░░░░░        ▒▒▓▓██▓▓▓▓▓▓▓▓▓▓██▓▓▓▓▓▓    ░░░░░░░░░░░░░░        
#           ░░░░░░░░░░░░  ██████░░░░░░░░░░░░░░░░  ░░░░░░████░░░░░░░░░░░░░░        
#           ░░░░░░░░░░████  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░████░░░░░░░░          
#                   ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ██                
#               ░░██    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██              
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#               ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██            
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  ░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#       ▓▓░░░░░░████░░▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░░░░░░░▒▒▓▓    
#       ██░░▒▒▓▓▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██▒▒░░██▒▒▒▒▒▒▒▒▒▒██░░░░░░░░░░░░░░▓▓░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░▒▒██▒▒▒▒▒▒██░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░░░░░██████▒▒░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░██░░░░░░░░░░██    
#       ██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░▒▒▓▓░░██░░░░░░░░░░██    
#       ██░░░░░░░░▒▒░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░██░░░░██░░░░░░░░░░██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▓▓▓▓▓▓▓▓░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓██░░░░██▓▓▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▓▓▒▒▒▒██▓▓░░░░░░▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒████▒▒▒▒████████████░░░░░░░░░░░░░░░░░░░░████████████▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██  ██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▓▓  ░░██░░░░░░░░░░░░░░░░░░░░██░░░░██▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒████░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▓▓██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██████░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░██▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██▒▒▒▒██████░░░░░░░░░░░░░░░░░░░░██████▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████▒▒▒▒▒▒▒▒██░░░░████████████████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒▒▒██░░░░░░██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████░░░░░░██████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓░░    
#       ░░██▒▒▒▒██▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓██░░░░░░██▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    


In [14]:
def negative_downsample(df, target, neg_frac = .1):
    # Negative downsampling
    df_pos = df[df[target]==1]
    df_neg = df[df[target]==0]
    
    df_neg = df_neg.sample(int(df.shape[0] * neg_frac), random_state=42)
    return pd.concat([df_pos,df_neg]).sort_index()

In [5]:
dtypes = {'label': 'int8',
 'uid': 'int32',
 'task_id': 'int16',
 'adv_id': 'int16',
 'creat_type_cd': 'int8',
 'adv_prim_id': 'int16',
 'dev_id': 'int8',
 'inter_type_cd': 'int8',
 'slot_id': 'int8',
 'spread_app_id': 'int8',
 'tags': 'int8',
 'app_first_class': 'int8',
 'app_second_class': 'int8',
 'age': 'int8',
 'city': 'int16',
 'city_rank': 'int8',
 'device_name': 'int8',
 'device_size': 'int16',
 'career': 'int8',
 'gender': 'int8',
 'net_type': 'int8',
 'residence': 'int8',
 'his_app_size': 'int8',
 'his_on_shelf_time': 'int8',
 'app_score': 'int8',
 'emui_dev': 'int8',
 'list_time': 'int8',
 'device_price': 'int8',
 'up_life_duration': 'int8',
 'up_membership_grade': 'int8',
 'membership_life_duration': 'int8',
 'consume_purchase': 'int8',
 'communication_onlinerate': 'object',
 'communication_avgonline_30d': 'int8',
 'indu_name': 'int8',
 'pt_d': 'int8'}

In [15]:
train_df  = pd.read_csv('train_data/train_data.csv', sep="|",dtype = dtypes ).reset_index()
test_df = pd.read_csv('test_data_A.csv', sep="|",dtype = dtypes ).reset_index()

cnt = CountEncoder(cols=['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','slot_id','spread_app_id','indu_name'])
cnt.fit(pd.concat([train_df.drop('label',axis=1),test_df.drop('id',axis=1)]))
train_m = cnt.transform(train_df.drop('label',axis=1))
test_m = cnt.transform(test_df.drop('id',axis=1))

In [17]:
train_m['label'] = train_df.label

In [18]:
FEATS = ['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','slot_id','spread_app_id','indu_name','age']

PARAMS = {  'learning_rate':0.05, 'max_bin': 100, 'max_depth': -1, 'min_child_samples': 5000,
        'n_estimators': 200,
        'num_leaves': 9,'objective': 'binary', 'n_jobs': -1,'subsample':.7,'boost_from_average': False}

In [20]:
train_downded = negative_downsample(train_m,'label')
train_data = lgb.Dataset(train_downded.drop('label',axis=1)[FEATS], label=train_downded.label, free_raw_data=True)
aa = lgb.train(PARAMS,train_data)
preds = aa.predict(test_m[FEATS])


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [22]:
test_df['probability']= preds

In [23]:
test_df[['id','probability']].set_index('id').to_csv('submission.csv')